# 뉴스 감성분석 model POC

- 모델 출처:[snunlp/KR-FinBert · Hugging Face](https://huggingface.co/snunlp/KR-FinBert)
- train 데이터셋 출처:
    1. [finance_sentiment_corpus/finance_data.csv at main · ukairia777/finance_sentiment_corpus          (github.com)](https://github.com/ukairia777/finance_sentiment_corpus/blob/main/finance_data.csv) 
    2. [통계청_인공지능 학습을 위한 고용기사 감성지수 라벨링 데이터_20201229 | 공공데이터포털 (data.go.kr)](https://www.data.go.kr/data/15075840/fileData.do)

---
## 사용 모델: krfinbert-model -hugging face

### 파인튜닝을 위한 train data set:

1. finance sentiment dataset - github
2. 고용기사 감성 지수 라벨링 dataset - 공공데이터포털

## 파인튜닝 개요

먼저 dataset을 불러오고 가져온 dataset을 학습에 사용할 수 있게 3가지 sentiment(긍정,부정,중립)들을 각각 1(긍정), 0(중립), -1(부정)으로 라벨링해주었다. 또 finetuning에 쓰일 Dataset을 모델이 사용할 수 있는 데이터로 전처리하는  함수(ReviewDataset 함수)를 만들고 가져온 dataset을 dataloader를 통해  batch 단위로 모델에 사용할 수 있게 가져온 뒤 사전학습 모델을 받아와 Finetuning 해주는 NET 클래스를 구현하였다. 그리고 구현한 NET 클래스를 통해 실제로 Load한 dataset을 학습시켜줬다.

finetuning 코드 설명

In [ ]:
# 0. 필요한 모듈 불러오기
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
import torch
from tqdm.auto import tqdm
import numpy as np

In [107]:
# 1. 데이터 불러오기
df = pd.read_csv(DICT_PATH+r"/df_news_sentiment",encoding= 'utf-8')

In [111]:
# 2.데이터 전처리

sent_list=[] #긍정 부정 중립을 각 2 0 -1로 바꿔서 담아줌
for sent in df['sentiment']:
    if sent =='positive':
        sent_list.append(2)

    elif sent =='neutral':
        sent_list.append(1)

    else:
        sent_list.append(0)        

    

In [112]:
df['sentiment'] = sent_list

In [113]:
df

,기사,sentiment
0,"Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로...",1
1,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...,1
2,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ...",0
3,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...,2
4,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4...",2
...,...,...
12459,코로나 충격에 따른 근로 시간 감소_ 과거 위기의 5배,0
12460,7월 사업체종사자 13만8천명 감소 코로나19 여파에 5개월째 감소,0
12461,한치 앞도 안보인다..채용문 꽁꽁 닫는 카드사,0
12462,토스_ 고객 상담직 신입 및 경력 30명 공개 채용,2


In [114]:
# 데이터의 각sentiment 별 비율 분포 확인 
df['sentiment'].value_counts()

sentiment
2    4870
0    4007
1    3587
Name: count, dtype: int64

In [115]:
#POC를 위해 train과 target데이터에서 10개 정도만 분리해서 구현이 가능한지만 확인
train=df['기사'][:10].to_numpy()
target = df["sentiment"][:10].to_numpy(dtype=np.float32).reshape(-1,1) #(-1,1)을 써서 [0,1,0....,1] 이런식인 target 배열을 [[0][1][0]...[1]]

In [116]:
#3.Dataset을 불러오고 모델이 사용할수 있는 형태로 전처리해주는 함수 구현

class ReviewDataset(torch.utils.data.Dataset): #이 데이터셋을 반복적으로 사용해 데이터프레임의 리뷰들을 하나하나 체크
    def __init__(self ,tokenizer , x, y = None ):
        self.tokenizer = tokenizer
        self.x = x #입력값
        self.y = y #정답
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self, idx):
        item = {}
        item["x"] = self.__tokenizer(self.x[idx]) #리뷰 각각을 토크나이저 해주기 위한 코드
        if self.y is not None: #정답이 있다면
            item["y"] = torch.Tensor(self.y[idx])#그 리뷰의 정답을 텐서로 변환
        return item #토큰화된 리뷰와 그에 맞는 텐서 데이터인 정답을 함께 반환
    def __tokenizer(self,text):
        inputs = self.tokenizer(text, add_special_tokens=True,padding='max_length', truncation=True,max_length = 37)
        for k, v in inputs.items():
            #k는 input_ids와 token_type_ids 그리고 어텐션마스크이며 v는 그 k(키)의 각 항목에 따른 v(벨류)를 저장
            #v는 각 값을 저장한 리스트
            inputs[k] = torch.LongTensor(v) #longtensor로 바꿔준 value값들을 inputs[k]로 저장
        return inputs

In [117]:
#4.사전학습 모델 선언
# Load model directly krfinbert 모델 파인튜닝 위해서 갖고옴
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_name = "snunlp/KR-FinBert-SC"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [118]:
#5.각 토큰화된 뉴스들을 dt 변수에 담아 dl 변수에 담아줌
dt = ReviewDataset(tokenizer,train,target) #리뷰데이터셋 객체 각 행(리뷰)들을 토큰화하고 정답 매칭해서 저장
dl = torch.utils.data.DataLoader(dt, batch_size=1,shuffle=False) #데이터 로더: 데이터셋 객체를 받아 미니배치형태로 구현하는 제네레이터
batch = next(iter(dl))
batch #이게 리뷰하나를 얘기하는 것

{'x': {'input_ids': tensor([[    2,    43,  5328, 10178,  5051, 10053,    16,  2078, 17805, 14243,
          10384,  8455,  8494, 14275,  8482,    16,  8703,  9716,  5008,  9873,
           5016,  3625,  5131,  9714,  5022, 10019,  8544,    18,     3,     0,
              0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])},
 'y': tensor([[1.]])}

In [119]:
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[    2,    43,  5328, 10178,  5051, 10053,    16,  2078, 17805, 14243,
          10384,  8455,  8494, 14275,  8482,    16,  8703,  9716,  5008,  9873,
           5016,  3625,  5131,  9714,  5022, 10019,  8544,    18,     3,     0,
              0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])},
 'y': tensor([[1.]])}

In [120]:
batch["x"].keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [121]:
outputs = model(**batch["x"]) #batch x 의 3가지 key타입들을 dict로 model에 넣어줌
outputs.keys()

odict_keys(['logits'])

In [122]:
#6.파인튜닝(NET)/ 모델 구현
class Net(torch.nn.Module):
    #def __init__(self, model_name):
    #    super().__init__()
    #    self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
    #    self.output_layer = torch.nn.Linear(self.model.config.hidden_size, 1)

    def __init__(self,model_name): #밑에서 module 받아야 하니
        super().__init__()
        model_name = "snunlp/KR-FinBert-SC"
        self.__tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.__model = AutoModelForSequenceClassification.from_pretrained(model_name)
        self.__model.to(DEVICE)
        #self.__softmax = F.softmax 일단은 주석 처리 손실함수때문

    def forward(self, text): #여기서 text를 받아 토큰화 시키고 output을 받으며 probs로 비선형화후 제일높은 확률의 친구를 pred에 넣어서 반환
        #x = self.model(**x)
        #x = self.output_layer(x[1])
        #text는 이미 token화 돼있음
        #inputs = self.__tokenizer(text, return_tensors='pt').to(DEVICE) 이미 토큰화돼있으므로
        #outputs = self.__model(**inputs)
        outputs= self.__model(**text) #토큰화된 text의 key들을 모델에 넘겨줌
        probs=outputs.logits
        ##print(probs)
        ##probs = self.__softmax(outputs.logits, dim=1) #output logit의 3개의 값을 활성화함수에 넣어줌
        ##print(probs.argmax().item())
        ##print(self.__model.config.id2label)
        #pred = self.__model.config.id2label[probs.argmax().item()].keys() #id2label 은 답이 담겨있는 dict 0:negative 1:neutral 2:positive
        
        #pred=pred.to(torch.long)
        #pred=torch.tensor(pred) 요거 써야됨
        #pred=torch.tensor(pred).reshape(1,1)
        #따라서 배치사이즈 1로 해서 한번 할 때마다 3개 중에 하나 고르게 하기 또한 pred에는 positive(str)이 저장되므로 value로 바꿔줌
        #잠깐 어차피 확률 3개중 제일 높은게 item이라면 계속 넣어주면되는거 아님
        return probs
        #return x

In [123]:
model = Net(model_name) #버트 베이스드 모델을 이용해 추가 학습 시키는것
model

Net(
  (_Net__model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(20000, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias

In [124]:
#7.target data를 원핫인코딩 해주는 함수 구현
def ohe_batch(batch): #batch['y']를 원핫인코딩해주는 함수 [-1.]은  [0,0,1] [0.]-> [0,1,0] [1.] -> [0,0,1]
    # tensor 0. =>
    if batch == torch.tensor([[0.]]):
        batch =torch.tensor([[1,0,0]])
        return batch 
        
    elif batch == torch.tensor([[1.]]):
        batch = torch.tensor([[0,1,0]])
        return batch 
    else:
        batch = torch.tensor([[0,0,1]]) 
        return batch
    

In [ ]:
#8.train/test 함수 구현

In [136]:
def train_loop(dataloader,model,loss_fn,optimizer,device):
    epoch_loss = 0
    model.train()
    for batch in tqdm(dataloader):
        pred = model(batch["x"].to(device))
        
        target_batch_ohe =ohe_batch(batch['y'])
        #loss = loss_fn(pred, batch["y"].to(device))
        
        target_batch_ohe = torch.argmax(target_batch_ohe, dim=1) # crossentropy를 쓸때는 원핫인코딩 벡터대신 정수형(long) 인덱스를 넘겨주어야함
        target_batch_ohe = target_batch_ohe.to(torch.long)
        #원핫인코딩된 인덱스의 번호(위치)가 저장
        
        loss = loss_fn(pred, target_batch_ohe.to(device))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)

    return epoch_loss

In [151]:
@torch.no_grad()
def test_loop(dataloader,model,loss_fn,device):
    epoch_loss = 0
    model.eval()

    pred_list = []
    sig = torch.nn.Sigmoid()

    for batch in tqdm(dataloader):

        pred = model(batch["x"].to(device))
        if batch.get("y") is not None:
            target_batch_ohe =ohe_batch(batch['y'])
            #loss = loss_fn(pred, batch["y"].to(device))
            #print(target_batch_ohe,1) 
            #print(pred,2)
            target_batch_ohe = torch.argmax(target_batch_ohe, dim=1) # crossentropy를 쓸때는 원핫인코딩 벡터대신 정수형(long) 인덱스를 넘겨주어야함
            target_batch_ohe = target_batch_ohe.to(torch.long)
            loss = loss_fn(pred, target_batch_ohe.to(device))

            epoch_loss += loss.item()

        pred = sig(pred) #시그모이드 함수에 넣어 선형->비선형
        pred = pred.to("cpu").numpy()
        
        pred_list.append(pred)

    epoch_loss /= len(dataloader)

    pred = np.concatenate(pred_list)
    print('pred:',pred)
    
    return epoch_loss , pred

In [144]:
#10.학습을 위한 하이퍼파라미터 정의
n_splits = 5
epochs = 4 #20
batch_size = 1
loss_fn = torch.nn.CrossEntropyLoss() #다중 분류 하려면 크로스엔트로피LOSS를 사용해야함 SOFTMAX가 안에서 이루어지므로 하면 안됨
SEED=42

In [145]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

cv = KFold(n_splits=n_splits,shuffle=True, random_state=SEED)

In [ ]:
#KFold 방식으로 train data와 validation dataset을 나누어주고 실제로 학습을 진행
is_holdout = True
#reset_seeds(SEED)
best_score_list = []
for i,(tri,vai) in enumerate(cv.split(train)): #kfold로 나누어준 곳에서 train과 validation set 순서대로 뽑아줌

    model = Net(model_name).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters())

    train_dt = ReviewDataset(tokenizer,train[tri],target[tri])
    valid_dt = ReviewDataset(tokenizer,train[vai],target[vai])
    train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True)
    valid_dl = torch.utils.data.DataLoader(valid_dt, batch_size=batch_size,shuffle=False)
    best_score = 0
    patience = 0

    for epoch in range(epochs):

        train_loss = train_loop(train_dl, model, loss_fn,optimizer,DEVICE )
    
        valid_loss , pred = test_loop(valid_dl, model, loss_fn,DEVICE  ) #test_loop에서 개같이 반환해주고 있으
        pred = (pred > 0.5).astype(int)
        print('벨리데이션 정답값:',target[vai],'예측값:',pred) #pred가 왜 1 0 1로 나올까
        score = accuracy_score(target[vai],pred )
        
        patience += 1
        print(train_loss,valid_loss,score,sep="\t")
        if best_score < score:
            patience = 0
            best_score = score
            torch.save(model.state_dict(),f"model_{i}.pth")

        if patience == 3:
            break

    best_score_list.append(best_score)

    if is_holdout:
        break